In [1]:
import os
import json
import glob
import logging
from tqdm import tqdm
from collections import defaultdict

# Set up logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('transform_evaluation_files.log'),
        logging.StreamHandler()
    ]
)

def transform_file(input_file_path, output_dir, stats):
    """
    Transform a single JSON file into question-organized format
    """
    try:
        # Read the input file
        with open(input_file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        
        # Get the base filename without extension
        base_filename = os.path.splitext(os.path.basename(input_file_path))[0]
        
        # Initialize output dictionary
        output_data = {}
        
        # Process each question
        for question_key, question_data in data.items():
            if isinstance(question_data, dict) and 'statement' in question_data and 'label' in question_data:
                # Transform label to eval_databases format
                label = question_data['label'].lower().strip()
                if label == 'yes':
                    eval_value = 'hallucination'
                elif label == 'no':
                    eval_value = 'fact'
                else:
                    eval_value = 'N/A'
                
                # Update statistics
                stats['total'][eval_value] += 1
                stats['by_question'][question_key][eval_value] += 1
                
                # Create question structure with both keys
                output_data[question_key] = {
                    "answer": question_data['statement'],
                    "eval_databases": eval_value
                }
        
        # Save combined file
        output_path = os.path.join(output_dir, f"{base_filename}.json")
        with open(output_path, 'w', encoding='utf-8') as f:
            json.dump(output_data, f, indent=2, ensure_ascii=False)
        
        logging.info(f"✅ Transformed {base_filename}.json -> question-organized format")
        return True
        
    except Exception as e:
        logging.error(f"❌ Error processing {input_file_path}: {str(e)}")
        return False

def print_statistics(stats):
    """
    Print detailed statistics about facts and hallucinations
    """
    print("\n" + "=" * 60)
    print("📊 STATISTICS SUMMARY")
    print("=" * 60)
    
    # Total statistics
    total_facts = stats['total']['fact']
    total_hallucinations = stats['total']['hallucination']
    total_na = stats['total']['N/A']
    total_all = total_facts + total_hallucinations + total_na
    
    print(f"🔢 TOTAL ACROSS ALL FILES:")
    print(f"   ✅ Facts: {total_facts}")
    print(f"   ❌ Hallucinations: {total_hallucinations}")
    print(f"   ❓ N/A: {total_na}")
    print(f"   📋 Total: {total_all}")
    
    if total_all > 0:
        fact_percentage = (total_facts / total_all) * 100
        hallucination_percentage = (total_hallucinations / total_all) * 100
        print(f"   📈 Fact Rate: {fact_percentage:.1f}%")
        print(f"   📉 Hallucination Rate: {hallucination_percentage:.1f}%")
    
    # By question statistics
    print(f"\n📋 BY QUESTION:")
    print("-" * 60)
    print(f"{'Question':<10} {'Facts':<8} {'Halluc.':<8} {'N/A':<6} {'Total':<8} {'Fact %':<8}")
    print("-" * 60)
    
    # Sort questions naturally (Q1, Q2, ..., Q10, Q11, ...)
    sorted_questions = sorted(stats['by_question'].keys(), 
                            key=lambda x: int(x[1:]) if x[1:].isdigit() else 999)
    
    for question in sorted_questions:
        q_stats = stats['by_question'][question]
        q_facts = q_stats['fact']
        q_halluc = q_stats['hallucination']
        q_na = q_stats['N/A']
        q_total = q_facts + q_halluc + q_na
        
        if q_total > 0:
            q_fact_pct = (q_facts / q_total) * 100
            print(f"{question:<10} {q_facts:<8} {q_halluc:<8} {q_na:<6} {q_total:<8} {q_fact_pct:<7.1f}%")
        else:
            print(f"{question:<10} {q_facts:<8} {q_halluc:<8} {q_na:<6} {q_total:<8} {'N/A':<8}")
    
    print("=" * 60)

def process_folder(input_folder, output_dir):
    """
    Process all JSON files in the input folder
    """
    try:
        # Create output directory if it doesn't exist
        os.makedirs(output_dir, exist_ok=True)
        
        # Initialize statistics
        stats = {
            'total': defaultdict(int),
            'by_question': defaultdict(lambda: defaultdict(int))
        }
        
        # Find all JSON files in the input folder
        json_files = glob.glob(os.path.join(input_folder, "*.json"))
        
        if not json_files:
            logging.warning(f"No JSON files found in {input_folder}")
            return
        
        logging.info(f"Found {len(json_files)} JSON files to process")
        
        # Process each file
        successful = 0
        failed = 0
        
        for json_file in tqdm(json_files, desc="Processing files"):
            if transform_file(json_file, output_dir, stats):
                successful += 1
            else:
                failed += 1
        
        # Print processing summary
        logging.info(f"Processing complete!")
        logging.info(f"✅ Successfully processed: {successful} files")
        logging.info(f"❌ Failed to process: {failed} files")
        logging.info(f"📁 Output files saved to: {output_dir}")
        
        # Print detailed statistics
        print_statistics(stats)
        
    except Exception as e:
        logging.error(f"Error processing folder: {str(e)}")

def main():
    # Configuration
    input_folder = "Database_dependent_evaluation/Clinical_trials/4-Evaluation/Evaluation_counterfactual/Gpt5/Paragraph_level/Evaluation_title_Paragraph_GPT"  # Replace with your input folder path
    output_dir =   "Database_dependent_evaluation/Clinical_trials/5-Labels/Inference_files_with_labels_Counterfactual/Gpt5/Paragraph_level/Inference_title_Paragraph"  # Replace with your output folder path
    
    # Print configuration
    print("🔄 Evaluation Files Transformer")
    print("=" * 60)
    print(f"📁 Input folder: {input_folder}")
    print(f"📁 Output folder: {output_dir}")
    print("🔄 Output format:")
    print("   {")
    print('     "Q1": {')
    print('       "answer": "statement text...",')
    print('       "eval_databases": "fact/hallucination"')
    print('     },')
    print('     "Q2": {')
    print('       "answer": "statement text...",')
    print('       "eval_databases": "fact/hallucination"')
    print('     }')
    print("   }")
    print("🔄 Transformation rules:")
    print("   - statement → answer")
    print("   - label 'no' → eval_databases = 'fact'")
    print("   - label 'yes' → eval_databases = 'hallucination'")
    print("   - other labels → eval_databases = 'N/A'")
    print("=" * 60)
    
    # Process the folder
    process_folder(input_folder, output_dir)

if __name__ == "__main__":
    main() 